# Wedge Query Assessment

Please be good

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
%load_ext google.cloud.bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/jackc/OneDrive/Documents/MSBA/Applied Data Analytics/Wedge/cloughwedge2021-3249989ff592.json"

In [3]:
client = bigquery.Client(project = 'cloughwedge2021')
gbq_proj_id = "cloughwedge2021"

In [ ]:
projID = "cloughwedge2021."
database = "transactions"

### Total Row Count

In [5]:
%%bigquery
SELECT count(*) as cnt
FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.81s/rows]


,cnt
0,85760139


The GBQ instance has a total row count of 85760139

### January/October 2012 Row Counts

In [13]:
%%bigquery
SELECT EXTRACT(Year from datetime) as Year,
       EXTRACT(Month from datetime) as Month,
       count(*)
FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 

GROUP BY Year, Month
HAVING Year = 2012 
ORDER BY Year, Month

Downloading: 100%|███████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.97rows/s]


,Yr,Mo,f0_
0,2012,1,1070907
1,2012,2,1034261
2,2012,3,1130248
3,2012,4,1135000
4,2012,5,1132624
5,2012,6,1053912
6,2012,7,1042457
7,2012,8,1029592
8,2012,9,1044436
9,2012,10,1042287


January has a row count of 1070907
October has a row count of 1025340

### Rows by Month

In [15]:
%%bigquery
SELECT EXTRACT(Month from datetime) as Mo,
       count(*) as cnt
FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 
WHERE EXTRACT(Year from datetime) <= 2016
GROUP BY Mo
ORDER BY cnt

Downloading: 100%|███████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.10rows/s]


,Mo,cnt
0,2,6556770
1,12,6740733
2,11,6955365
3,9,6975855
4,8,7008748
5,6,7012157
6,7,7013013
7,1,7056762
8,10,7085752
9,4,7417388


January has a row count of 7056762
October has a row 7085752

### Null Counts

In [16]:
%%bigquery
SELECT COUNTIF(trans_subtype is NULL) as Null_TS,
       COUNTIF(datetime is NULL) as Null_DT,
       COUNTIF(local IS NULL) as Null_Local,
       COUNTIF(card_no IS NULL) as Null_CN
FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.83s/rows]


,Null_TS,Null_DT,Null_Local,Null_CN
0,7123792,0,234843,0


transaction subtypes has 7123792 null values
datetimes has 0 null values
locals has 234843 null values
card numbers has 0 null values

### High Volume Cards

In [17]:
%%bigquery
SELECT card_no, 
       COUNT(*) as cnt
FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 
WHERE card_no != 3
GROUP BY card_no
ORDER BY cnt desc
LIMIT 10

Downloading: 100%|███████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.30rows/s]


,card_no,cnt
0,12539.0,2161167
1,10499.0,1013697
2,14140.0,942230
3,20074.0,643460
4,14987.0,460630
5,10504.0,164864
6,21517.0,133006
7,23170.0,99276
8,19750.0,65876
9,15876.0,59053


__What card is #5 on the list and how many rows does it have?__
Card_No 14987, it has 460630 rows

### Owner ID "18736" Counts 

In [18]:
%%bigquery
SELECT card_no, 
       COUNT(*) as cnt
FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 
WHERE card_no = 18736
GROUP BY card_no

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.01s/rows]


,card_no,cnt
0,18736.0,12153


__How many rows do I have?__
12153

### Popular Products

In [20]:
%%bigquery
SELECT LOWER(Description), 
       COUNT(*) as cnt
FROM `cloughwedge2021.cloughwedge2021.transArchive_*`
WHERE trans_type = "I" AND
      Description != "Discount"
GROUP BY LOWER(Description)
ORDER BY cnt desc
LIMIT 10

Downloading: 100%|███████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.22rows/s]


,f0_,cnt
0,banana organic,908639
1,green patch redemption,572473
2,wedge cookie,510140
3,avocado hass organic,456771
4,broccoli organic,344657
5,celery organic,254479
6,citrus lemon organic,246058
7,salad mix organic,225088
8,pepper bell red organic,224162
9,spinach bulk organic,209826


__Which product is #1? How many rows does it have?__ 
Organic bananas, which has 908639 rows

__Which product is #4? How many rows does it have?__
Organic Hass avocados, which has 456771 rows

### Single Record Items

In [22]:
%%bigquery
SELECT 
    COUNT(DISTINCT Description) as SingleRecordItems
FROM (
    SELECT LOWER(Description) as Description,
           COUNT(*) as cnt
    FROM `cloughwedge2021.cloughwedge2021.transArchive_*` 
  WHERE trans_type = "I" AND Description != "Discount"
    GROUP BY Description
    HAVING cnt = 1
    )

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/rows]


,SingleRecordItems
0,2769


There are 2769 single record items

### Owner Fractions By Year

In [23]:
%%bigquery
SELECT
  Y,
  OwnerRows,
  NonOwnerRows,
  ROUND(OwnerRows/(OwnerRows+NonOwnerRows),4) AS OwnerFrac
FROM (
  SELECT
    EXTRACT(Year
    FROM
      datetime) AS Y,
    COUNTIF(card_no != 3) AS OwnerRows,
    COUNTIF(card_no = 3) AS NonOwnerRows
  FROM
    `cloughwedge2021.cloughwedge2021.transArchive_*` 
  GROUP BY
    Y)
ORDER BY
  Y

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.59rows/s]


,Y,OwnerRows,NonOwnerRows,OwnerFrac
0,2010,9005825,3128483,0.7422
1,2011,8980792,3200841,0.7372
2,2012,9442984,3287078,0.7418
3,2013,9214660,3103828,0.7480
4,2014,9237995,2932261,0.7591
5,2015,9217601,3183007,0.7433
6,2016,8166872,2721171,0.7501
7,2017,703766,232975,0.7513


__Which year has the highest portion of owner rows? What year? What fraction?__
2014 with a fraction of .7591

__Which year has the lowest portion of owner rows? What year? What fraction?__
2011 with a fraction of .7372